In [ ]:
import pandas as pd
from diffusers import DiffusionPipeline
import torch

In [ ]:
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", 
    torch_dtype=torch.float16, variant="fp16",
    use_safetensors=True
)
base.enable_model_cpu_offload()
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.enable_model_cpu_offload()

n_steps = 100
high_noise_frac = 0.9

def generate_image(word):
    prompt = f"one {word}, minimalist, watercolor, only one subject, white background, preset cartoon"
    image = base(
        prompt=prompt,
        num_inference_steps=n_steps,
        denoising_end=high_noise_frac,
        output_type="latent",
    ).images
    image = refiner(
        prompt=prompt,
        num_inference_steps=n_steps,
        denoising_start=high_noise_frac,
        image=image,
    ).images[0]
    image.save(f'{word}.png')
    return


In [ ]:
words = pd.read_csv('100words.csv',sep=';',header=None)

for word in words[0]:
     generate_image(word)